- https://ai.google.dev/gemini-api/docs/thinking?hl=zh-cn
- 关于 thinking 部分的说明
    - https://blog.google/technology/google-deepmind/google-gemini-updates-io-2025/#developer-experience
    - Thought summaries
        - 2.5 Pro and Flash will now include thought summaries in the Gemini API and in Vertex AI. Thought summaries take the **model’s raw thoughts and organize them into a clear format with headers, key details and information about model actions**, like when they use tools.
        - We hope that with a more structured, streamlined format on the model’s thinking process, developers and users will find the interactions with Gemini models easier to understand and debug.

### chat messages

- history
    - system prompt

### thinking & streaming & token count 

```python
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# 确保您的 API 密钥已通过环境变量或直接在 Client 中配置
# client = genai.Client(api_key="YOUR_API_KEY")

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

prompt = """
Alice, Bob, and Carol each live in a different house on the same street: red, green, and blue.
The person who lives in the red house owns a cat.
Bob does not live in the green house.
Carol owns a dog.
The green house is to the left of the red house.
Alice does not own a cat.
Who lives in each house, and what pet do they own?
"""

prompt = """hello"""

thoughts = ""
answer = ""
# 创建一个变量来存储最终的元数据
final_usage_metadata = None

# 获取流式响应的迭代器
response_stream = client.models.generate_content_stream(
    model="gemini-2.5-pro",
    contents=prompt,
    config=types.GenerateContentConfig(
      thinking_config=types.ThinkingConfig(
        include_thoughts=True
      )
    )
)

print("--- 流式输出 ---")
for chunk in response_stream:
  # 从每个数据块中捕获 usage_metadata。
  # 循环结束后，该变量将持有最后一个数据块的元数据，其中包含最终总数。
  if chunk.usage_metadata:
      final_usage_metadata = chunk.usage_metadata

  for part in chunk.candidates[0].content.parts:
    if not part.text:
      continue
    elif part.thought:
      if not thoughts:
        print("\nThoughts summary:")
      print(part.text, end="", flush=True)
      thoughts += part.text
    else:
      if not answer:
        print("\nAnswer:")
      print(part.text, end="", flush=True)
      answer += part.text

print("\n\n--- 词元统计 ---")
if final_usage_metadata:
    # 'thoughts_token_count' 是模型内部原始思考过程消耗的词元数 [citation:3]
    print(f"思考部分的词元数 (Thinking Tokens): {final_usage_metadata.thoughts_token_count}")
    
    # 'candidates_token_count' 是最终生成的回答部分的词元数 [citation:1][citation:3]
    print(f"回答部分的词元数 (Answer Tokens): {final_usage_metadata.candidates_token_count}")
    
    # 'prompt_token_count' 是您输入提示的词元数 [citation:1]
    print(f"提示部分的词元数 (Prompt Tokens): {final_usage_metadata.prompt_token_count}")
    
    # 'total_token_count' 是以上所有部分的总和 [citation:1]
    print(f"总词元数 (Total Tokens): {final_usage_metadata.total_token_count}")
else:
    print("未能获取 usage_metadata。")


```

### langgraph with gemini

- https://ai.google.dev/gemini-api/docs/langgraph-example?hl=zh-cn